In [1]:

"""
Professional Random Password Generator
Optimized for Google Colab with Gradio Interface
Features: Advanced security rules, customization, strength analysis, batch generation
"""

# Install required packages (uncomment if needed)
# !pip install gradio pyperclip

import gradio as gr
import random
import string
import secrets
import re
from datetime import datetime
from typing import List, Tuple

class PasswordGenerator:
    """Advanced password generator with security rules and customization"""

    def __init__(self):
        self.lowercase = string.ascii_lowercase
        self.uppercase = string.ascii_uppercase
        self.digits = string.digits
        self.symbols = "!@#$%^&*()_+-=[]{}|;:,.<>?"
        self.ambiguous_chars = "il1Lo0O"

    def calculate_entropy(self, password: str, charset_size: int) -> float:
        """Calculate password entropy in bits"""
        import math
        if len(password) == 0:
            return 0
        return len(password) * math.log2(charset_size)

    def check_password_strength(self, password: str) -> Tuple[str, str, int]:
        """Analyze password strength and return rating, color, and score"""
        score = 0
        feedback = []

        # Length check
        if len(password) >= 16:
            score += 30
        elif len(password) >= 12:
            score += 20
        elif len(password) >= 8:
            score += 10
        else:
            feedback.append("Password too short")

        # Character diversity
        if re.search(r'[a-z]', password):
            score += 15
        if re.search(r'[A-Z]', password):
            score += 15
        if re.search(r'\d', password):
            score += 15
        if re.search(r'[!@#$%^&*()_+\-=\[\]{}|;:,.<>?]', password):
            score += 25

        # Determine strength
        if score >= 80:
            strength = "🔒 Very Strong"
            color = "#10b981"  # green
        elif score >= 60:
            strength = "✓ Strong"
            color = "#3b82f6"  # blue
        elif score >= 40:
            strength = "⚠ Moderate"
            color = "#f59e0b"  # orange
        else:
            strength = "✗ Weak"
            color = "#ef4444"  # red

        return strength, color, score

    def generate_password(
        self,
        length: int = 16,
        use_lowercase: bool = True,
        use_uppercase: bool = True,
        use_digits: bool = True,
        use_symbols: bool = True,
        exclude_ambiguous: bool = False,
        exclude_chars: str = "",
        must_include_all: bool = True,
        min_lowercase: int = 0,
        min_uppercase: int = 0,
        min_digits: int = 0,
        min_symbols: int = 0
    ) -> Tuple[str, str, str]:
        """Generate a secure password with specified criteria"""

        # Validate inputs
        if length < 4:
            return "", "❌ Error: Password length must be at least 4 characters", ""

        if not any([use_lowercase, use_uppercase, use_digits, use_symbols]):
            return "", "❌ Error: Select at least one character type", ""

        # Build character set
        charset = ""
        if use_lowercase:
            charset += self.lowercase
        if use_uppercase:
            charset += self.uppercase
        if use_digits:
            charset += self.digits
        if use_symbols:
            charset += self.symbols

        # Remove ambiguous characters if requested
        if exclude_ambiguous:
            charset = ''.join(c for c in charset if c not in self.ambiguous_chars)

        # Remove custom excluded characters
        if exclude_chars:
            charset = ''.join(c for c in charset if c not in exclude_chars)

        if not charset:
            return "", "❌ Error: No characters available with current settings", ""

        # Calculate minimum required length
        min_required = min_lowercase + min_uppercase + min_digits + min_symbols
        if min_required > length:
            return "", f"❌ Error: Minimum requirements ({min_required}) exceed password length ({length})", ""

        # Generate password with cryptographically secure random
        max_attempts = 100
        for attempt in range(max_attempts):
            password_list = []

            # Add minimum required characters
            if use_lowercase and min_lowercase > 0:
                valid_chars = [c for c in self.lowercase if c in charset]
                password_list.extend(secrets.choice(valid_chars) for _ in range(min_lowercase))

            if use_uppercase and min_uppercase > 0:
                valid_chars = [c for c in self.uppercase if c in charset]
                password_list.extend(secrets.choice(valid_chars) for _ in range(min_uppercase))

            if use_digits and min_digits > 0:
                valid_chars = [c for c in self.digits if c in charset]
                password_list.extend(secrets.choice(valid_chars) for _ in range(min_digits))

            if use_symbols and min_symbols > 0:
                valid_chars = [c for c in self.symbols if c in charset]
                password_list.extend(secrets.choice(valid_chars) for _ in range(min_symbols))

            # Fill remaining length
            remaining = length - len(password_list)
            password_list.extend(secrets.choice(charset) for _ in range(remaining))

            # Shuffle to avoid predictable patterns
            random.shuffle(password_list)
            password = ''.join(password_list)

            # Verify password meets requirements
            if must_include_all:
                valid = True
                if use_lowercase and not any(c in self.lowercase for c in password):
                    valid = False
                if use_uppercase and not any(c in self.uppercase for c in password):
                    valid = False
                if use_digits and not any(c in self.digits for c in password):
                    valid = False
                if use_symbols and not any(c in self.symbols for c in password):
                    valid = False

                if valid:
                    break
            else:
                break

        # Calculate strength
        strength, color, score = self.check_password_strength(password)
        entropy = self.calculate_entropy(password, len(charset))

        # Create detailed analysis
        analysis = f"""
### 📊 Password Analysis

**Strength:** <span style="color: {color}; font-weight: bold; font-size: 1.2em;">{strength}</span>

**Metrics:**
- Length: {len(password)} characters
- Character Pool Size: {len(charset)} characters
- Entropy: {entropy:.1f} bits
- Security Score: {score}/100

**Composition:**
- Lowercase: {'✓' if any(c in self.lowercase for c in password) else '✗'}
- Uppercase: {'✓' if any(c in self.uppercase for c in password) else '✗'}
- Digits: {'✓' if any(c in self.digits for c in password) else '✗'}
- Symbols: {'✓' if any(c in self.symbols for c in password) else '✗'}

**Recommendations:**
{self._get_recommendations(password, length, score)}
"""

        return password, "✅ Password generated successfully!", analysis

    def _get_recommendations(self, password: str, length: int, score: int) -> str:
        """Generate security recommendations"""
        recommendations = []

        if length < 12:
            recommendations.append("• Consider using at least 12 characters for better security")
        if score < 60:
            recommendations.append("• Add more character types (uppercase, symbols) for stronger password")
        if not re.search(r'[!@#$%^&*()_+\-=\[\]{}|;:,.<>?]', password):
            recommendations.append("• Include special symbols for maximum security")

        if not recommendations:
            recommendations.append("• Excellent! This password meets high security standards")
            recommendations.append("• Store it in a secure password manager")
            recommendations.append("• Never reuse this password across different accounts")

        return '\n'.join(recommendations) if recommendations else "Password meets security best practices!"

    def generate_batch(
        self,
        count: int,
        length: int,
        use_lowercase: bool,
        use_uppercase: bool,
        use_digits: bool,
        use_symbols: bool,
        exclude_ambiguous: bool,
        exclude_chars: str
    ) -> str:
        """Generate multiple passwords"""
        if count < 1 or count > 50:
            return "❌ Error: Count must be between 1 and 50"

        passwords = []
        for i in range(count):
            pwd, status, _ = self.generate_password(
                length=length,
                use_lowercase=use_lowercase,
                use_uppercase=use_uppercase,
                use_digits=use_digits,
                use_symbols=use_symbols,
                exclude_ambiguous=exclude_ambiguous,
                exclude_chars=exclude_chars,
                must_include_all=True
            )
            if pwd:
                strength, _, _ = self.check_password_strength(pwd)
                passwords.append(f"{i+1}. {pwd}  [{strength}]")

        if not passwords:
            return "❌ Error: Could not generate passwords with current settings"

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        header = f"🔐 Batch Password Generation - {count} passwords\n"
        header += f"Generated: {timestamp}\n"
        header += f"Length: {length} characters\n"
        header += "=" * 60 + "\n\n"

        return header + '\n'.join(passwords)

# Initialize generator
generator = PasswordGenerator()

# Custom CSS for professional styling
custom_css = """
.container {
    max-width: 1200px;
    margin: auto;
}
.password-output {
    font-family: 'Courier New', monospace;
    font-size: 1.3em;
    font-weight: bold;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    text-align: center;
    letter-spacing: 2px;
    word-break: break-all;
}
.status-message {
    font-size: 1.1em;
    padding: 10px;
    border-radius: 5px;
}
.title-custom {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.5em;
    font-weight: bold;
    text-align: center;
    margin-bottom: 10px;
}
"""

# Create Gradio interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        <div class="title-custom">
        🔐 Professional Password Generator
        </div>
        <p style="text-align: center; font-size: 1.1em; color: #666;">
        Generate cryptographically secure passwords with advanced customization options
        </p>
        """
    )

    with gr.Tabs():
        # Tab 1: Single Password Generation
        with gr.Tab("🎯 Generate Password"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Basic Settings")
                    length_slider = gr.Slider(
                        minimum=4,
                        maximum=64,
                        value=16,
                        step=1,
                        label="Password Length",
                        info="Recommended: 16+ characters"
                    )

                    gr.Markdown("### 🔤 Character Types")
                    use_lowercase = gr.Checkbox(label="Lowercase (a-z)", value=True)
                    use_uppercase = gr.Checkbox(label="Uppercase (A-Z)", value=True)
                    use_digits = gr.Checkbox(label="Digits (0-9)", value=True)
                    use_symbols = gr.Checkbox(label="Symbols (!@#$...)", value=True)

                    gr.Markdown("### 🎛️ Advanced Options")
                    exclude_ambiguous = gr.Checkbox(
                        label="Exclude Ambiguous Characters (i, l, 1, L, o, 0, O)",
                        value=False,
                        info="Improves readability"
                    )
                    exclude_chars = gr.Textbox(
                        label="Exclude Specific Characters",
                        placeholder="e.g., @#$ or any characters to exclude",
                        info="Enter characters to exclude from generation"
                    )
                    must_include_all = gr.Checkbox(
                        label="Must Include All Selected Types",
                        value=True,
                        info="Ensures at least one character from each selected type"
                    )

                    with gr.Accordion("🔒 Minimum Requirements", open=False):
                        min_lowercase = gr.Slider(0, 10, 0, step=1, label="Min Lowercase")
                        min_uppercase = gr.Slider(0, 10, 0, step=1, label="Min Uppercase")
                        min_digits = gr.Slider(0, 10, 0, step=1, label="Min Digits")
                        min_symbols = gr.Slider(0, 10, 0, step=1, label="Min Symbols")

                    generate_btn = gr.Button("🎲 Generate Password", variant="primary", size="lg")

                with gr.Column(scale=1):
                    gr.Markdown("### 🔑 Generated Password")
                    password_output = gr.Textbox(
                        label="Your Password",
                        interactive=True,
                        show_copy_button=True,
                        elem_classes="password-output",
                        lines=2
                    )
                    status_output = gr.Markdown(label="Status")
                    analysis_output = gr.Markdown(label="Security Analysis")

                    gr.Markdown(
                        """
                        ### 💡 Quick Tips
                        - **Copy** your password using the copy button
                        - **Never reuse** passwords across accounts
                        - **Store** in a secure password manager
                        - **Change** regularly for critical accounts
                        """
                    )

        # Tab 2: Batch Generation
        with gr.Tab("📦 Batch Generation"):
            gr.Markdown("### Generate Multiple Passwords at Once")

            with gr.Row():
                with gr.Column():
                    batch_count = gr.Slider(
                        minimum=1,
                        maximum=50,
                        value=10,
                        step=1,
                        label="Number of Passwords",
                        info="Generate up to 50 passwords"
                    )
                    batch_length = gr.Slider(
                        minimum=8,
                        maximum=32,
                        value=16,
                        step=1,
                        label="Password Length"
                    )

                    with gr.Row():
                        batch_lowercase = gr.Checkbox(label="Lowercase", value=True)
                        batch_uppercase = gr.Checkbox(label="Uppercase", value=True)
                    with gr.Row():
                        batch_digits = gr.Checkbox(label="Digits", value=True)
                        batch_symbols = gr.Checkbox(label="Symbols", value=True)

                    batch_exclude_ambiguous = gr.Checkbox(
                        label="Exclude Ambiguous Characters",
                        value=False
                    )
                    batch_exclude_chars = gr.Textbox(
                        label="Exclude Specific Characters",
                        placeholder="Optional"
                    )

                    batch_generate_btn = gr.Button(
                        "🚀 Generate Batch",
                        variant="primary",
                        size="lg"
                    )

                with gr.Column():
                    batch_output = gr.Textbox(
                        label="Generated Passwords",
                        lines=20,
                        show_copy_button=True
                    )

        # Tab 3: Security Guide
        with gr.Tab("📚 Security Guide"):
            gr.Markdown(
                """
                # 🛡️ Password Security Best Practices

                ## Why Strong Passwords Matter
                Strong passwords are your first line of defense against unauthorized access. A weak password can be cracked in seconds, while a strong one could take millions of years.

                ## Characteristics of Strong Passwords

                ### ✅ DO:
                - **Use 16+ characters** for maximum security
                - **Mix character types**: uppercase, lowercase, numbers, symbols
                - **Make it random**: avoid dictionary words and personal information
                - **Use unique passwords** for each account
                - **Store in a password manager**: LastPass, 1Password, Bitwarden

                ### ❌ DON'T:
                - **Use personal information**: names, birthdays, addresses
                - **Use dictionary words**: even with substitutions (P@ssw0rd)
                - **Reuse passwords**: across multiple accounts
                - **Share passwords**: via email, text, or messaging apps
                - **Write them down**: in unsecure locations

                ## Password Strength Levels

                | Strength | Length | Crack Time | Use Case |
                |----------|--------|------------|----------|
                | 🔴 Weak | < 8 | Seconds | Never use |
                | 🟡 Moderate | 8-11 | Hours to Days | Low-security accounts |
                | 🟢 Strong | 12-15 | Years | Most accounts |
                | 🔵 Very Strong | 16+ | Centuries | Critical accounts |

                ## Additional Security Measures

                1. **Enable Two-Factor Authentication (2FA)** wherever possible
                2. **Use a Password Manager** to generate and store unique passwords
                3. **Update passwords regularly** for sensitive accounts
                4. **Monitor for breaches** using services like Have I Been Pwned
                5. **Use passkeys** when available (passwordless authentication)

                ## This Tool's Security Features

                - **Cryptographically Secure**: Uses Python's `secrets` module
                - **Customizable**: Tailor to specific security requirements
                - **Batch Generation**: Create multiple passwords efficiently
                - **Strength Analysis**: Real-time security assessment
                - **Entropy Calculation**: Mathematical security measurement

                ---

                ### 🔒 Remember: Your digital security is only as strong as your weakest password!
                """
            )

    # Event handlers
    generate_btn.click(
        fn=generator.generate_password,
        inputs=[
            length_slider,
            use_lowercase,
            use_uppercase,
            use_digits,
            use_symbols,
            exclude_ambiguous,
            exclude_chars,
            must_include_all,
            min_lowercase,
            min_uppercase,
            min_digits,
            min_symbols
        ],
        outputs=[password_output, status_output, analysis_output]
    )

    batch_generate_btn.click(
        fn=generator.generate_batch,
        inputs=[
            batch_count,
            batch_length,
            batch_lowercase,
            batch_uppercase,
            batch_digits,
            batch_symbols,
            batch_exclude_ambiguous,
            batch_exclude_chars
        ],
        outputs=[batch_output]
    )

# Launch the app
if __name__ == "__main__":
    app.launch(
        share=True,  # Creates a public link for sharing
        debug=False,
        show_error=True
    )
    print("\n🚀 Password Generator is running!")
    print("📱 Access the interface using the URL above")
    print("🔗 Share link will expire in 72 hours")

/tmp/ipython-input-1722241450.py:300: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as app:
/tmp/ipython-input-1722241450.py:300: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3973f8197e118fabad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🚀 Password Generator is running!
📱 Access the interface using the URL above
🔗 Share link will expire in 72 hours
